In [206]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys
boxer_list = ['Deontay Wilder']
def get_boxers(boxers):
    page_link = 'http://beta.compuboxdata.com/fighter'
    chromedriver = 'C:\\Users\\User\\Downloads\\chromedriver'
    cdriver = webdriver.Chrome(chromedriver)
    cdriver.maximize_window()
    cdriver.get(page_link)
    wait = WebDriverWait(cdriver,10)
    wait.until(EC.visibility_of_element_located((By.ID,'s2id_autogen1'))).send_keys(str(boxers))
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'select2-result-label'))).click()
    while True:
        try:
            element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'view_more')))
            element.click()
        except TimeoutException:
            break
    fighters =  cdriver.find_elements_by_xpath("//div[@class='row row-bottom-margin-5']/div[2]")
    cols = ['fighters', 'stats']
    fight_data = pd.DataFrame(columns = cols)
    for fighter in fighters:
        fight_data.append({'fighters':fighter.text},ignore_index=True)
        cdriver.find_element_by_xpath("//a[contains(@onclick,'get_fight_report')]").click()
        punches = cdriver.find_elements_by_xpath("//div[@class='modal-content']")
        for punch in punches:
            fight_data.append({'stats':punch.text},ignore_index=True)
    cdriver.refresh()
    return fight_data
boxing_dict = {boxer.replace(' ',''):pd.DataFrame([get_boxers(boxer)]) for boxer in boxer_list}


ElementClickInterceptedException: Message: element click intercepted: Element <a class="view_more" href="javascript:void(0);" onclick="_search('0')"></a> is not clickable at point (274, 774). Other element would receive the click: <div class="col-xs-12 col-sm-12 col-md-12 col-lg-12">...</div>
  (Session info: chrome=78.0.3904.108)


In [208]:
#testing on one boxer
page_link = 'http://beta.compuboxdata.com/fighter'
chromedriver = 'C:\\Users\\User\\Downloads\\chromedriver'
cdriver = webdriver.Chrome(chromedriver)
cdriver.maximize_window()
cdriver.get(page_link)
wait = WebDriverWait(cdriver,10)
wait.until(EC.visibility_of_element_located((By.ID,'s2id_autogen1'))).send_keys('Deontay Wilder')
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'select2-result-label'))).click()
while True:
    try:
        element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'view_more')))
        element.click()
    except TimeoutException:
        break
fighters =  cdriver.find_elements_by_xpath("//div[@class='row row-bottom-margin-5']/div[2]")
cols = ['fighters', 'stats']
fight_data = pd.DataFrame(columns = cols)
for fighter in fighters:
    fight_data.append({'fighters':fighter.text},ignore_index=True)
    elem = wait.until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@onclick,'get_fight_report')]")))
    elem.click()
    punches = cdriver.find_elements_by_xpath("//div[@class='modal-content']")
    for punch in punches:
        fight_data.append({'stats':punch.text},ignore_index=True)